<a href="https://colab.research.google.com/github/spniroul/bayesian-optimization/blob/main/BayesianOpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python Script

In [ ]:
!pip install bayesian-optimization

Phase 1:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from bayes_opt import BayesianOptimization

#Loading MNIST Dataset
mnist_test = pd.read_csv('sample_data/mnist_test.csv')
Y_test = mnist_test.iloc[:, 0].values
X_test = mnist_test.iloc[:, 1:].values

mnist_train = pd.read_csv('sample_data/mnist_train_small.csv')
Y_train = mnist_train.iloc[:, 0].values
X_train = mnist_train.iloc[:, 1:].values

#Normalizing pixel values
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#Objective Function for BO
def obj_func(alpha, learning_rate):
    model = MLPClassifier(
        hidden_layer_sizes=(64,),  #Fixed Hidden Layer
        activation="relu",
        alpha=alpha,
        learning_rate_init=learning_rate,
        max_iter=30,
        random_state=7,
    )
    model.fit(X_train, Y_train)
    return model.score(X_test, Y_test)

#Parameter Bounds
bounds = {
    "alpha": (1e-5, 0.1),
    "learning_rate": (1e-4, 0.1),
}

#Running Optimizer
optimizer = BayesianOptimization(f=obj_func, pbounds=bounds, random_state=7)
optimizer.maximize(init_points=2, n_iter=5)

#Extracting parameter trace
params_df = pd.DataFrame([res["params"] for res in optimizer.res])
params_df["Iteration"] = range(1, len(params_df) + 1)

#Normalizing parameter values
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
normalized_params = scaler.fit_transform(params_df[["alpha", "learning_rate"]])
normalized_df = pd.DataFrame(normalized_params, columns=["alpha", "learning_rate"])
normalized_df["Iteration"] = params_df["Iteration"]

#Plotting parameter traces
plt.figure(figsize=(10, 6))
for param in ["alpha", "learning_rate"]:
    plt.plot(normalized_df["Iteration"], normalized_df[param], marker='o', label=param)

plt.xlabel("Iterations")
plt.ylabel("Normalized Parameter Values")
plt.title("Parameter Traces")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

#Plotting convergence
iterations = list(range(1, len(optimizer.space.target) + 1))
targets = optimizer.space.target

plt.figure(figsize=(8, 5))
plt.plot(iterations, targets, marker='o')
plt.xlabel("Iterations")
plt.ylabel("Accuracy")
plt.title("Convergence")
plt.grid(True)
plt.tight_layout()
plt.show()

#Print results
print("Best Parameters: ", optimizer.max["params"])
print("Best Accuracy: ", optimizer.max["target"])


Phase 2:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from bayes_opt import BayesianOptimization

#Loading MNIST Dataset
mnist_test = pd.read_csv('sample_data/mnist_test.csv')
Y_test = mnist_test.iloc[:,0].values
X_test = mnist_test.iloc[:,1:].values

mnist_train = pd.read_csv('sample_data/mnist_train_small.csv')
Y_train = mnist_train.iloc[:,0].values
X_train = mnist_train.iloc[:,1:].values

#Normalizing pixel values
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#Objective Function for BO
def obj_func(alpha, learning_rate, hidden_layer_size):
    model = MLPClassifier(
        hidden_layer_sizes=(int(hidden_layer_size),),
        activation="relu",
        alpha=alpha,
        learning_rate_init=learning_rate,
        max_iter=30,
        random_state=7,
    )
    model.fit(X_train, Y_train)
    return model.score(X_test, Y_test)

#Parameter Bounds
bounds = {
    "alpha": (1e-5, 0.1),
    "learning_rate": (1e-4, 0.1),
    "hidden_layer_size": (32, 128),
}

#Running Optimizer
optimizer = BayesianOptimization(f=obj_func, pbounds=bounds, random_state=7)
optimizer.maximize(init_points=2, n_iter=5)

#Extracting parameter trace
params_df = pd.DataFrame([res["params"] for res in optimizer.res])
params_df["Iteration"] = range(1, len(params_df) + 1)

#Normalizing parameter values

scaler = MinMaxScaler()
normalized_params = scaler.fit_transform(params_df[["alpha", "learning_rate", "hidden_layer_size"]])
normalized_df = pd.DataFrame(normalized_params, columns=["alpha", "learning_rate", "hidden_layer_size"])
normalized_df["Iteration"] = params_df["Iteration"]

#Plotting all traces in one plot
plt.figure(figsize=(10, 6))
for param in ["alpha", "learning_rate", "hidden_layer_size"]:
    plt.plot(normalized_df["Iteration"], normalized_df[param], marker='o', label=param)

plt.xlabel("Iterations")
plt.ylabel("Normalized Parameter Values")
plt.title("Parameter Traces")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

#Plotting Convergence
iterations = list(range(1, len(optimizer.space.target) + 1))
targets = optimizer.space.target

plt.figure(figsize=(8, 5))
plt.plot(iterations, targets, marker='o')
plt.xlabel("Iterations")
plt.ylabel("Accuracy")
plt.title("Convergence")
plt.grid(True)
plt.tight_layout()
plt.show()

#Print Results
print("Best Parameters: ", optimizer.max["params"])
print("Best Accuracy: ", optimizer.max["target"])